### This scripts takes about 1 1/2 minutes seconds to execute

In [ ]:
from tqdm.notebook import tqdm, trange
from importlib import reload
import horseracinglib
reload(horseracinglib)
from horseracinglib import *
import pandas as pd
from IPython.display import display, HTML
pd.options.display.max_columns = None

# Load test data (not seen by models) - Legacy MLR/trivial models

In [ ]:
runners = pd.read_csv('data\\runners_test.csv', parse_dates=['meeting_date'], infer_datetime_format=True)
runners.sort_values(by=['race_id', 'runner_id'], inplace=True)

races = pd.read_csv('data\\races_test.csv', parse_dates=['meeting_date'], infer_datetime_format=True)
races.sort_values(by=['race_id'], inplace=True)

runners = runners.merge(races[["race_id", "course_Kempton", "course_Lingfield", "course_Southwell", "course_Wolverhampton", "class_1", "class_2", "class_3", "class_4", "class_5", "gt1mi"]], on='race_id', how='left')
runners.head()

In [ ]:
runners_single_race = runners[runners.race_id == 500816]
runners_single_race

# Load test data (not seen by models) - New NN models

In [ ]:
# note, test data is model-specific

test_data = torch.load('data\\mktprob_test_data.pt')
test_data.runners_wide.head()

In [ ]:
test_data.races.head()

In [ ]:
test_data.runners_wide.loc[runners_single_race.race_id.iloc[0], :]

In [ ]:
test_data.races.loc[runners_single_race.race_id.iloc[0], :]

# Demonstrate calculating model probabilities for single race - Legacy MLR/trivial models

In [ ]:
AlunOwen_multinomial_logit = MultinomialLogitModel('models\\AlunOwen_multinomial_logit_coefficients.csv', model_prefix='AO3')
display(AlunOwen_multinomial_logit.model_coefficients)
AlunOwen_multinomial_logit.calculate_model_probabilities_for_single_race(runners_single_race)

In [ ]:
MG1_multinomial_logit = MultinomialLogitModel('models\\MG1_multinomial_logit_coefficients.csv', model_prefix='MG1')
display(MG1_multinomial_logit.model_coefficients)
MG1_multinomial_logit.calculate_model_probabilities_for_single_race(runners_single_race)

In [ ]:
Amalgum_multinomial_logit = MultinomialLogitModel('models\\Amalgum_multinomial_logit_coefficients.csv', model_prefix='Amalgum')
display(Amalgum_multinomial_logit.model_coefficients)
Amalgum_multinomial_logit.calculate_model_probabilities_for_single_race(runners_single_race)

In [ ]:
my_random_choice = RandomChoiceModel(model_prefix='RC')
my_random_choice.calculate_model_probabilities_for_single_race(runners_single_race)

In [ ]:
my_favourite_choice = FavouriteChoiceModel(model_prefix='FC')
my_favourite_choice.calculate_model_probabilities_for_single_race(runners_single_race)

# Demonstrate calculating model probabilities for single race - New NN models

In [ ]:
model_inventory = {}
for model_prefix, model_filename, test_data_filename in [('MP', 'models\\mktprob.pt', 'data\\mktprob_test_data.pt'), ('AO3', 'models\\AlunOwen_v3.pt', 'data\\AlunOwen_v3_test_data.pt')]:
    test_data = torch.load(test_data_filename)
    model_inventory[model_prefix] = NeuralNetworkModel(model_filename, test_data, model_prefix)
    display(model_inventory[model_prefix].model_object)
    display(model_inventory[model_prefix].calculate_model_probabilities_for_single_race(runners_single_race))

# Demonstrate calculating model probabilities for multiple races

In [ ]:
AlunOwen_multinomial_logit.calculate_model_probabilities_for_multiple_races(runners)
AlunOwen_multinomial_logit.model_probabilities.head()

In [ ]:
MG1_multinomial_logit.calculate_model_probabilities_for_multiple_races(runners)
MG1_multinomial_logit.model_probabilities.head()

In [ ]:
MG1_multinomial_logit.calculate_model_probabilities_for_multiple_races(runners)
MG1_multinomial_logit.model_probabilities.head()

In [ ]:
my_random_choice.calculate_model_probabilities_for_multiple_races(runners)
my_random_choice.model_probabilities.head()

In [ ]:
my_favourite_choice.calculate_model_probabilities_for_multiple_races(runners)
my_favourite_choice.model_probabilities.head()

In [ ]:
model_inventory['MP'].calculate_model_probabilities_for_multiple_races(runners)
model_inventory['MP'].model_probabilities.head()

In [ ]:
model_inventory['AO3'].calculate_model_probabilities_for_multiple_races(runners)
model_inventory['AO3'].model_probabilities.head()

# Calculate model accuracy

In [ ]:
# All of the following code was written by ChatGPT-4

# Create an empty DataFrame
df = pd.DataFrame(columns=['Model', 'Accuracy'])

for model, model_prefix in [(AlunOwen_multinomial_logit, 'AO3'), 
                            (MG1_multinomial_logit, 'MG1'),
                            (Amalgum_multinomial_logit, 'Amalgum'),
                            (my_random_choice, 'RC'), 
                            (my_favourite_choice, 'FC')]:
    accuracy = model.calculate_model_accuracy(runners)
    df = pd.concat([df, pd.DataFrame([{'Model': model_prefix, 'Accuracy': f'{accuracy * 100:.1f}%'}])], ignore_index=True)

# Display the DataFrame as an HTML table
display(HTML(df.to_html(index=False)))

# Demonstrate probability model assessment

In [ ]:
my_model_assessment = ProbabilityModelAssessment([AlunOwen_multinomial_logit, MG1_multinomial_logit, Amalgum_multinomial_logit, model_inventory['MP']], runners[['race_id', 'runner_id', 'stall_number', 'win', 'adj_mkt_prob']])
display(my_model_assessment.assessment)
my_model_assessment.perform_assessment()

In [ ]:
my_model_assessment.show_diagnostic_plots()

# Demonstrate calculating wagering strategy payoffs for single race

In [ ]:
def pick_hi_mod_prob(augmented_runners_single_race):
    stakes = pd.Series(np.full(len(augmented_runners_single_race), 0, np.float64), index=augmented_runners_single_race.index)
    max_mod_prob = np.max(augmented_runners_single_race.mod_prob)
    i = augmented_runners_single_race[augmented_runners_single_race.mod_prob == max_mod_prob].index[0]
    stakes[i] = 1
    return stakes

def kelly_criterion_top_two_favourites(augmented_runners_single_race):
    stakes = pd.Series(np.full(len(augmented_runners_single_race), 0, np.float64), index=augmented_runners_single_race.index)
    sorted_runners = augmented_runners_single_race.sort_values('mod_prob', ascending=False)
    top_two_indices = sorted_runners.index[:2]
    
    for i in top_two_indices:
        p = augmented_runners_single_race.loc[i, 'mod_prob']
        q = 1 - p
        b = (1 / p) - 1  # odds
        f = max((b * p - q) / b, 0)  # Kelly Criterion formula, but we don't want to bet negative amounts
        stakes[i] = f

    return stakes

def pick_hi_mod_prob_class_1(augmented_runners_single_race):
    stakes = pd.Series(np.full(len(augmented_runners_single_race), 0, np.float64), index=augmented_runners_single_race.index)
    max_mod_prob = np.max(augmented_runners_single_race.mod_prob)
    i = augmented_runners_single_race[augmented_runners_single_race.mod_prob == max_mod_prob].index[0]
    stakes[i] = augmented_runners_single_race.class_1[i]
    return stakes

def pick_hi_mod_prob_class_2(augmented_runners_single_race):
    stakes = pd.Series(np.full(len(augmented_runners_single_race), 0, np.float64), index=augmented_runners_single_race.index)
    max_mod_prob = np.max(augmented_runners_single_race.mod_prob)
    i = augmented_runners_single_race[augmented_runners_single_race.mod_prob == max_mod_prob].index[0]
    stakes[i] = augmented_runners_single_race.class_2[i]
    return stakes

def pick_hi_mod_prob_class_3(augmented_runners_single_race):
    stakes = pd.Series(np.full(len(augmented_runners_single_race), 0, np.float64), index=augmented_runners_single_race.index)
    max_mod_prob = np.max(augmented_runners_single_race.mod_prob)
    i = augmented_runners_single_race[augmented_runners_single_race.mod_prob == max_mod_prob].index[0]
    stakes[i] = augmented_runners_single_race.class_3[i]
    return stakes

def pick_hi_mod_prob_class_4(augmented_runners_single_race):
    stakes = pd.Series(np.full(len(augmented_runners_single_race), 0, np.float64), index=augmented_runners_single_race.index)
    max_mod_prob = np.max(augmented_runners_single_race.mod_prob)
    i = augmented_runners_single_race[augmented_runners_single_race.mod_prob == max_mod_prob].index[0]
    stakes[i] = augmented_runners_single_race.class_4[i]
    return stakes

def pick_hi_mod_prob_class_5(augmented_runners_single_race):
    stakes = pd.Series(np.full(len(augmented_runners_single_race), 0, np.float64), index=augmented_runners_single_race.index)
    max_mod_prob = np.max(augmented_runners_single_race.mod_prob)
    i = augmented_runners_single_race[augmented_runners_single_race.mod_prob == max_mod_prob].index[0]
    stakes[i] = augmented_runners_single_race.class_5[i]
    return stakes


In [ ]:
# my_1st_strategy = WageringStrategy(my_favourite_choice, pick_hi_mod_prob, 'PickHiModProb')
# my_1st_strategy = WageringStrategy(my_favourite_choice, kelly_criterion_top_two_favourites, 'KCTopTwo')
# my_1st_strategy = WageringStrategy(my_favourite_choice, pick_hi_mod_prob_class_1, 'PickHiModProbClass1')
my_1st_strategy = WageringStrategy(Amalgum_multinomial_logit, pick_hi_mod_prob_class_1, 'PickHiModProbClass1')
my_1st_strategy.calculate_strategy_stakes_and_payoffs_for_single_race(runners_single_race)

In [ ]:
# my_2nd_strategy = WageringStrategy(Amalgum_multinomial_logit, pick_hi_mod_prob, 'PickHiModProb')
# my_2nd_strategy = WageringStrategy(Amalgum_multinomial_logit, kelly_criterion_top_two_favourites, 'KCTopTwo')
# my_2nd_strategy = WageringStrategy(my_favourite_choice, pick_hi_mod_prob_class_2, 'PickHiModProbClass2')
my_2nd_strategy = WageringStrategy(Amalgum_multinomial_logit, pick_hi_mod_prob_class_2, 'PickHiModProbClass2')
my_2nd_strategy.calculate_strategy_stakes_and_payoffs_for_single_race(runners_single_race)

In [ ]:
# my_3rd_strategy = WageringStrategy(AlunOwen_multinomial_logit, pick_hi_mod_prob, 'PickHiModProb')
# my_3rd_strategy = WageringStrategy(AlunOwen_multinomial_logit, kelly_criterion_top_two_favourites, 'KCTopTwo')
# my_3rd_strategy = WageringStrategy(my_favourite_choice, pick_hi_mod_prob_class_3, 'PickHiModProbClass3')
my_3rd_strategy = WageringStrategy(Amalgum_multinomial_logit, pick_hi_mod_prob_class_3, 'PickHiModProbClass3')
my_3rd_strategy.calculate_strategy_stakes_and_payoffs_for_single_race(runners_single_race)

In [ ]:
# my_4th_strategy = WageringStrategy(MG1_multinomial_logit, pick_hi_mod_prob, 'PickHiModProb')
# my_4th_strategy = WageringStrategy(MG1_multinomial_logit, kelly_criterion_top_two_favourites, 'KCTopTwo')
# my_4th_strategy = WageringStrategy(my_favourite_choice, pick_hi_mod_prob_class_4, 'PickHiModProbClass4')
my_4th_strategy = WageringStrategy(Amalgum_multinomial_logit, pick_hi_mod_prob_class_4, 'PickHiModProbClass4')
my_4th_strategy.calculate_strategy_stakes_and_payoffs_for_single_race(runners_single_race)

In [ ]:
# my_5th_strategy = WageringStrategy(my_random_choice, pick_hi_mod_prob, 'PickHiModProb')
# my_5th_strategy = WageringStrategy(my_random_choice, kelly_criterion_top_two_favourites, 'KCTopTwo')
# my_5th_strategy = WageringStrategy(my_favourite_choice, pick_hi_mod_prob_class_5, 'PickHiModProbClass5')
my_5th_strategy = WageringStrategy(Amalgum_multinomial_logit, pick_hi_mod_prob_class_5, 'PickHiModProbClass5')
my_5th_strategy.calculate_strategy_stakes_and_payoffs_for_single_race(runners_single_race)

# Demonstrate calculating wagering strategy payoffs for multiple races

In [ ]:
my_1st_strategy.calculate_strategy_stakes_and_payoffs_for_multiple_races(runners)
my_1st_strategy.strategy_stakes_and_payoffs.head()

In [ ]:
my_2nd_strategy.calculate_strategy_stakes_and_payoffs_for_multiple_races(runners)
my_2nd_strategy.strategy_stakes_and_payoffs.head()

In [ ]:
my_3rd_strategy.calculate_strategy_stakes_and_payoffs_for_multiple_races(runners)
my_3rd_strategy.strategy_stakes_and_payoffs.head()

In [ ]:
my_4th_strategy.calculate_strategy_stakes_and_payoffs_for_multiple_races(runners)
my_4th_strategy.strategy_stakes_and_payoffs.head()

In [ ]:
my_5th_strategy.calculate_strategy_stakes_and_payoffs_for_multiple_races(runners)
my_5th_strategy.strategy_stakes_and_payoffs.head()

# Demonstrate wagering strategy assessment

In [ ]:
my_strategy_assessment = WageringStrategyAssessment([my_1st_strategy, my_2nd_strategy, my_3rd_strategy, my_4th_strategy, my_5th_strategy], runners[['race_id', 'meeting_date', 'runner_id', 'stall_number', 'win', 'adj_mkt_prob']])
display(my_strategy_assessment.assessment)
my_strategy_assessment.perform_assessment()

In [ ]:
my_strategy_assessment.plot_cumulative_return()